In [2]:
! pip install kaggle

In [3]:
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle competitions download -c fake-news

 53% 5.00M/9.42M [00:00<00:00, 6.20MB/s]
100% 9.42M/9.42M [00:00<00:00, 10.8MB/s]
  0% 0.00/40.6k [00:00<?, ?B/s]
100% 40.6k/40.6k [00:00<00:00, 42.2MB/s]
 89% 33.0M/37.0M [00:01<00:00, 22.6MB/s]
100% 37.0M/37.0M [00:01<00:00, 36.4MB/s]


In [7]:
from zipfile import ZipFile
df='/content/train.csv.zip'
with ZipFile (df,'r') as zip:
  zip.extractall()
  print('extraction done') 

extraction done


Data Attributes:-

id: unique id for a news article

title: the title of a news article

author: author of the news article

text: the text of the article; could be incomplete

label: a label that marks the article as potentially unreliable

1: unreliable

0: reliable

Import Dependencies


In [9]:
import numpy as np
import pandas as pd
import re   # Regular expression operations - matching strings of different types #finding patterns in the text 
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

**Knowing Stopwords:**

In [12]:
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Loading dataset

In [15]:
df=pd.read_csv('/content/train.csv')

In [16]:
df.shape

(20800, 5)

In [17]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


Checking for null values

In [19]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [21]:
# replacing the null values with empty string as we have a large dataset 
df=df.fillna('')

Merging the title and the author column for predictions

In [24]:
df['newcontent']=df['author']+''+df['title']

In [25]:
df['newcontent']

0        Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1        Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom...
2        Consortiumnews.comWhy the Truth Might Get You ...
3        Jessica Purkiss15 Civilians Killed In Single U...
4        Howard PortnoyIranian woman jailed for fiction...
                               ...                        
20795    Jerome HudsonRapper T.I.: Trump a ’Poster Chil...
20796    Benjamin HoffmanN.F.L. Playoffs: Schedule, Mat...
20797    Michael J. de la Merced and Rachel AbramsMacy’...
20798    Alex AnsaryNATO, Russia To Hold Parallel Exerc...
20799               David SwansonWhat Keeps the F-35 Alive
Name: newcontent, Length: 20800, dtype: object

In [26]:
df.head()

,id,title,author,text,label,newcontent
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.comWhy the Truth Might Get You ...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss15 Civilians Killed In Single U...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard PortnoyIranian woman jailed for fiction...


Separating label/target from the dataset

In [27]:
x=df.drop(columns='label', axis=1)
y=df['label']

In [29]:
print (x)


          id  ...                                         newcontent
0          0  ...  Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1          1  ...  Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom...
2          2  ...  Consortiumnews.comWhy the Truth Might Get You ...
3          3  ...  Jessica Purkiss15 Civilians Killed In Single U...
4          4  ...  Howard PortnoyIranian woman jailed for fiction...
...      ...  ...                                                ...
20795  20795  ...  Jerome HudsonRapper T.I.: Trump a ’Poster Chil...
20796  20796  ...  Benjamin HoffmanN.F.L. Playoffs: Schedule, Mat...
20797  20797  ...  Michael J. de la Merced and Rachel AbramsMacy’...
20798  20798  ...  Alex AnsaryNATO, Russia To Hold Parallel Exerc...
20799  20799  ...             David SwansonWhat Keeps the F-35 Alive

[20800 rows x 5 columns]


In [30]:
print (y)

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


**STEMMING**

--> reducing words and their plurals to root word

with the help of potterstemmer 




In [31]:
port_stem=PorterStemmer()

In [32]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content) # ^--> means remove every thing except a-z and A-z ,' ' with space from content,.sub=substituting 
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()  # split and convert to a list
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content



Applying stemming function to our newcontent column

In [33]:
df['newcontent']=df['newcontent'].apply(stemming)

In [34]:
print(df['newcontent'])

0        darrel lucushous dem aid even see comey letter...
1        daniel j flynnflynn hillari clinton big woman ...
2                consortiumnew comwhi truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoyiranian woman jail fiction unpub...
                               ...                        
20795    jerom hudsonrapp trump poster child white supr...
20796    benjamin hoffmann f l playoff schedul matchup ...
20797    michael j de la merc rachel abramsmaci said re...
20798    alex ansarynato russia hold parallel exercis b...
20799                        david swansonwhat keep f aliv
Name: newcontent, Length: 20800, dtype: object


--> we can observe that all the extra symbols other then a-z,A-Z 

have been removed from the newcontent column of our dataset like , : , 12 ...

and conversion to root word is done

Defining x and y values 

x = newcontent

y= target labels

In [35]:
   x=df['newcontent'].values
   y=df['label'].values

In [36]:
print(x)

['darrel lucushous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynnflynn hillari clinton big woman campu breitbart'
 'consortiumnew comwhi truth might get fire' ...
 'michael j de la merc rachel abramsmaci said receiv takeov approach hudson bay new york time'
 'alex ansarynato russia hold parallel exercis balkan'
 'david swansonwhat keep f aliv']


In [37]:
y

array([1, 0, 1, ..., 0, 1, 1])

In [38]:
y.shape

(20800,)

**Converting textual content to vectors** 

In [39]:
vector=TfidfVectorizer()
vector.fit(x)
x=vector.transform(x)

In [41]:
print(x)

  (0, 26340)	0.28088379401596425
  (0, 22724)	0.2552336018069161
  (0, 15019)	0.43006226759639316
  (0, 14555)	0.29177259684200296
  (0, 12782)	0.24619727512767195
  (0, 8022)	0.23133661742488731
  (0, 6273)	0.2839932825877813
  (0, 5969)	0.35488202138141456
  (0, 5006)	0.2472595823572816
  (0, 4211)	0.3625320323150658
  (0, 578)	0.2694167078545385
  (1, 27923)	0.36911845953845024
  (1, 11313)	0.24166773097712638
  (1, 8772)	0.5258635625386451
  (1, 5916)	0.31810058109638056
  (1, 4767)	0.23338756776626793
  (1, 3859)	0.45980466668763476
  (1, 3281)	0.18652439327549428
  (1, 2622)	0.3562953366945267
  (2, 26235)	0.3665032495181434
  (2, 16361)	0.43295215406038445
  (2, 9454)	0.30743020569262086
  (2, 8567)	0.3411947414020896
  (2, 5240)	0.40440534260277944
  (2, 5121)	0.5511414848555652
  :	:
  (20797, 25776)	0.08220218573989037
  (20797, 25319)	0.3119640221826561
  (20797, 22086)	0.24902354987792552
  (20797, 20778)	0.2729578683228216
  (20797, 20493)	0.249994989010826
  (20797, 17505

**Process of test train splitting**

In [45]:
x_train , x_test , y_train , y_test =train_test_split(x,y,test_size=0.2, stratify =y,random_state=2)
#stratify parameter will preserve the proportion of target as in original dataset, in the train and test datasets as well.

**Training the logistic regression model**

In [46]:
lr=LogisticRegression()  # using sigmoid function if value above a certain threshold than 1 else 0 
# for binary classification logistic regression is good

In [47]:
lr.fit(x_train,y_train)

LogisticRegression()

**Evaluating**

In [48]:
xtrain_prediction=lr.predict(x_train)

**Calculating Accuracy Score for training data**

In [49]:
train_accuracy=accuracy_score(xtrain_prediction,y_train)

In [50]:
print("accuracy score on training data is: ",train_accuracy)

accuracy score on training data is:  0.9719951923076923


**Calculating Accuracy Score for testing data**

In [51]:
xtest_prediction=lr.predict(x_test)

In [52]:
test_accuracy=accuracy_score(xtest_prediction,y_test)

In [53]:
print("accuracy score on test data is: ",test_accuracy)

accuracy score on test data is:  0.9548076923076924


**Checking the readiness of the predictive model**

In [78]:
for i in range(4):
  xnew=x_test[i]     # if i=0, taking the first row of the test data     
  prediction=lr.predict(xnew)
  p=print(prediction)

  if (prediction[0]==0):
    print (" the news is real ")
  else:
    print ("  fake  alert  !!!!! ")



  q=print(y_test[i])

  if(p==q):
    print("correct prediciton")
  else:
    print("incorrect prediction")





[1]
  fake  alert  !!!!! 
1
correct prediciton
[0]
 the news is real 
0
correct prediciton
[1]
  fake  alert  !!!!! 
1
correct prediciton
[0]
 the news is real 
0
correct prediciton


**Our model is predicting correctly !!!**